In [10]:
#!/usr/bin/env python3
import logging
from pathlib import Path

from Pegasus.api import *

logging.basicConfig(level=logging.DEBUG)

# Properties
props = Properties()
props["pegasus.data.configuration"] = "condorio"
props["dagman.retry"] = "5"
props.write()

#Replica
rc = ReplicaCatalog()\
        .add_replica("local", "saved_stated.txt", Path(".").resolve() / "saved_stated.txt")\
        .write()

# Transformation
check = Transformation(
        "print_nums.py",
        site="local",
        pfn=str(Path(".").resolve() / "print_nums.py"),
        is_stageable=True
    )

tc = TransformationCatalog().add_transformations(check).write()

# Workflow
job = Job(check)\
        .add_checkpoint(File("saved_stated.txt"), stage_out=True)\
        .set_stdout("output.txt")\
        .add_profiles(Namespace.PEGASUS, key="checkpoint.time", value=1)\
        .add_profiles(Namespace.PEGASUS, key="maxwalltime", value=2)


wf = Workflow("checkpoint-wf").add_jobs(job)

In [ ]:
try:
    wf.plan(submit=True)\
        .wait()\
        .analyze()\
        .statistics()
except PegasusClientError as e:
    print(e.output)

Plan:
[main] WARN  schema.JsonMetaSchema  - Unknown keyword $defs - you should define your own Meta Schema. If the keyword is irrelevant for validation, just use a NonValidationKeyword
[main] WARN  schema.JsonMetaSchema  - Unknown keyword additionalItems - you should define your own Meta Schema. If the keyword is irrelevant for validation, just use a NonValidationKeyword
[main] WARN  schema.JsonMetaSchema  - Unknown keyword examples - you should define your own Meta Schema. If the keyword is irrelevant for validation, just use a NonValidationKeyword
2020.07.20 21:45:41.256 UTC:    
2020.07.20 21:45:41.261 UTC:   ----------------------------------------------------------------------- 
2020.07.20 21:45:41.267 UTC:   File for submitting this DAG to HTCondor           : checkpoint-wf-0.dag.condor.sub 
2020.07.20 21:45:41.273 UTC:   Log of DAGMan debugging messages                 : checkpoint-wf-0.dag.dagman.out 
2020.07.20 21:45:41.278 UTC:   Log of HTCondor library output                

[############################################------]  88.9% ..Running (Completed: 8, Queued: 1, Running: 0, Failed: 0)